In [1]:
import polyline
import folium
import requests

In [2]:
def get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):
    
    #steps = "?steps=true"
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://192.168.219.101:5000//route/v1/driving/"
    r = requests.get(url + loc)#+steps) 
    if r.status_code!= 200:
        return {}
  
    res = r.json()   
    routes = polyline.decode(res['routes'][0]['geometry'])
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']
    
    out = {'route':routes,
           'start_point':start_point,
           'end_point':end_point,
           'distance':distance
          }

    return out

In [3]:
import pandas as pd
df = pd.read_csv('data/daejeon_data.csv',encoding='utf-8')

In [4]:
df = df[:10000]

In [5]:
#df
df['routes'] = df.apply(lambda x: get_route(x['dprt_x'], 
                                            x['dprt_y'],
                                            x['arrvl_x'], 
                                            x['arrvl_y']), axis=1)


In [7]:
df.head()

,cost,call_cost,night_add,dprt_time,dprt_x,dprt_y,arrvl_time,arrvl_x,arrvl_y,trv_dist,...,arrvl_grid_x,arrvl_grid_y,dprt_st_id,dprt_st_type,lag_duration,lead_duration,dprt_time_60min,arrvl_time_15min,arrvl_time_60min,routes
0,6240,0,1,2017-03-02T00:40:04Z,127.400625,36.324826,2017-03-02T00:49:09Z,127.426930,36.351128,4623,...,127.427554,36.354894,0,0,242,806,2017-03-02T00:00:00Z,2017-03-02T00:45:00Z,2017-03-02T00:00:00Z,{}
1,8040,0,1,2017-03-02T00:50:02Z,127.426183,36.351171,2017-03-02T01:03:28Z,127.455353,36.310763,6160,...,127.449874,36.309828,0,0,545,331,2017-03-02T00:00:00Z,2017-03-02T01:00:00Z,2017-03-02T01:00:00Z,{}
2,4320,0,1,2017-03-02T01:18:28Z,127.432046,36.332448,2017-03-02T01:23:59Z,127.437788,36.349416,2592,...,127.438707,36.345885,0,0,806,405,2017-03-02T01:00:00Z,2017-03-02T01:15:00Z,2017-03-02T01:00:00Z,{}
3,4200,0,1,2017-03-02T01:50:23Z,127.437773,36.350068,2017-03-02T01:57:08Z,127.449766,36.338551,2010,...,127.449857,36.336875,91,2,331,581,2017-03-02T01:00:00Z,2017-03-02T01:45:00Z,2017-03-02T01:00:00Z,{}
4,5520,0,1,2017-03-02T01:59:37Z,127.447768,36.337411,2017-03-02T02:09:18Z,127.449510,36.314553,3407,...,127.449868,36.318844,0,0,405,550,2017-03-02T01:00:00Z,2017-03-02T02:00:00Z,2017-03-02T02:00:00Z,{}


In [27]:
def get_time(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):
    
    #steps = "?steps=true"
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://127.0.0.1:5000/route/v1/driving/"
    r = requests.get(url + loc)#+steps) 
    if r.status_code!= 200:
        return {}
  
    res = r.json()   
    duration = res['routes'][0]['duration']
    #distance = res['routes'][0]['distance']
    return duration

In [120]:

df['timestamps'] = None
for j in range(df.shape[0]):
    duration_list = [0]
    df_routes = df['routes'][j]
    for i in range(1,len(df_routes['route'])):
        duration = get_time(df_routes['route'][i-1][1],df_routes['route'][i-1][0],df_routes['route'][i][1],df_routes['route'][i][0])
        time_stamps = duration_list[i-1] + duration
        duration_list.append(time_stamps)
    df['timestamps'][j] = duration_list

<ipython-input-120-641dc7851833>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timestamps'][j] = duration_list


In [211]:
df['timestamps'][:10000]

0       [0, 23.5, 84.6, 133.2, 223.7, 249.299999999999...
1       [0, 20.6, 35.3, 54.8, 98.8, 104.1, 125.3999999...
2       [0, 54.8, 108.5, 134.2, 154.7, 198.29999999999...
3       [0, 5.9, 33.9, 55.5, 95.9, 135.7, 147.29999999...
4       [0, 120.3, 187.3, 215.20000000000002, 231.3, 2...
                              ...                        
9995    [0, 6.9, 30.5, 38.6, 45.9, 66.0, 70.9, 92.0, 9...
9996    [0, 7.7, 35.7, 57.300000000000004, 61.2, 97.7,...
9997                          [0, 43, 51.4, 116.5, 144.7]
9998    [0, 38.3, 73.3, 237.0, 256.0, 284.1, 336.1, 36...
9999    [0, 117.2, 173.9, 199.20000000000002, 218.0000...
Name: timestamps, Length: 10000, dtype: object

In [121]:
df['coordinates']=None
for i in range(df.shape[0]):
    coordinates_list = []
    for j in range(len(df['routes'][i]['route'])):
        coord_list = []
        coord_list.append(df['routes'][i]['route'][j][1])
        coord_list.append(df['routes'][i]['route'][j][0])
        coordinates_list.append(coord_list)
    df['coordinates'][i] = coordinates_list

<ipython-input-121-3003c118ee9a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['coordinates'][i] = coordinates_list


In [214]:
df['coordinates']

0       [[127.40056, 36.32481], [127.39968, 36.32704],...
1       [[127.42618, 36.35116], [127.42813, 36.35148],...
2       [[127.43206, 36.33245], [127.43272, 36.33119],...
3       [[127.43769, 36.35004], [127.43798, 36.3495], ...
4       [[127.44786, 36.33738], [127.44286, 36.32946],...
                              ...                        
9995    [[127.44361, 36.34433], [127.44355, 36.3444], ...
9996    [[127.43761, 36.35021], [127.43798, 36.3495], ...
9997    [[127.42692, 36.32898], [127.42815, 36.32685],...
9998    [[127.42468, 36.32317], [127.42843, 36.32442],...
9999    [[127.38737, 36.3513], [127.40163, 36.35127], ...
Name: coordinates, Length: 10000, dtype: object

In [213]:
df.head()

,cost,call_cost,night_add,dprt_time,dprt_x,dprt_y,arrvl_time,arrvl_x,arrvl_y,trv_dist,...,lag_duration,lead_duration,dprt_time_60min,arrvl_time_15min,arrvl_time_60min,routes,timestamps,coordinates,duration,trip
0,6240,0,1,2017-03-02T00:40:04Z,127.400625,36.324826,2017-03-02T00:49:09Z,127.426930,36.351128,4623,...,242,806,2017-03-02T00:00:00Z,2017-03-02T00:45:00Z,2017-03-02T00:00:00Z,"{'route': [(36.32481, 127.40056), (36.32704, 1...","[0, 23.5, 84.6, 133.2, 223.7, 249.299999999999...","[[127.40056, 36.32481], [127.39968, 36.32704],...",476.4,1
1,8040,0,1,2017-03-02T00:50:02Z,127.426183,36.351171,2017-03-02T01:03:28Z,127.455353,36.310763,6160,...,545,331,2017-03-02T00:00:00Z,2017-03-02T01:00:00Z,2017-03-02T01:00:00Z,"{'route': [(36.35116, 127.42618), (36.35148, 1...","[0, 20.6, 35.3, 54.8, 98.8, 104.1, 125.3999999...","[[127.42618, 36.35116], [127.42813, 36.35148],...",452.3,2
2,4320,0,1,2017-03-02T01:18:28Z,127.432046,36.332448,2017-03-02T01:23:59Z,127.437788,36.349416,2592,...,806,405,2017-03-02T01:00:00Z,2017-03-02T01:15:00Z,2017-03-02T01:00:00Z,"{'route': [(36.33245, 127.43206), (36.33119, 1...","[0, 54.8, 108.5, 134.2, 154.7, 198.29999999999...","[[127.43206, 36.33245], [127.43272, 36.33119],...",324.2,3
3,4200,0,1,2017-03-02T01:50:23Z,127.437773,36.350068,2017-03-02T01:57:08Z,127.449766,36.338551,2010,...,331,581,2017-03-02T01:00:00Z,2017-03-02T01:45:00Z,2017-03-02T01:00:00Z,"{'route': [(36.35004, 127.43769), (36.3495, 12...","[0, 5.9, 33.9, 55.5, 95.9, 135.7, 147.29999999...","[[127.43769, 36.35004], [127.43798, 36.3495], ...",176.6,4
4,5520,0,1,2017-03-02T01:59:37Z,127.447768,36.337411,2017-03-02T02:09:18Z,127.449510,36.314553,3407,...,405,550,2017-03-02T01:00:00Z,2017-03-02T02:00:00Z,2017-03-02T02:00:00Z,"{'route': [(36.33738, 127.44786), (36.32946, 1...","[0, 120.3, 187.3, 215.20000000000002, 231.3, 2...","[[127.44786, 36.33738], [127.44286, 36.32946],...",380.7,5


In [122]:
df['duration'] = df['timestamps'].apply(lambda x:x[-1])

In [123]:
new_df = pd.DataFrame()
new_df['carnum'] = None
new_df['tpep_pickup_datetime'] = None
new_df['tpep_dropoff_datetime'] = None
new_df['pickup_longitude'] = None
new_df['pickup_latitude'] = None
new_df['dropoff_longitude'] = None
new_df['dropoff_latitude'] = None

In [130]:
from datetime import timedelta
for i in range(df.shape[0]):
    dprt_time = pd.to_datetime(df['dprt_time'][i])
    car_num = df['carnum'][i]
    timestamps = df['timestamps'][i]
    coordinates = df['coordinates'][i]
    for j in range(1,len(timestamps)):
        new_new = dprt_time + timedelta(seconds = timestamps[j-1])
        new_time = dprt_time + timedelta(seconds = timestamps[j])
        new_df = new_df.append(pd.DataFrame([[car_num,new_new, new_time,coordinates[j-1][0],coordinates[j-1][1],coordinates[j][0],coordinates[j][1]]], columns=['carnum','tpep_pickup_datetime', 'tpep_dropoff_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']), ignore_index=True)
    

In [223]:
new_df.head(20)

,carnum,tpep_pickup_datetime,tpep_dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip,seconds,altitude
0,대전50바1004,2017-03-02 00:40:04+00:00,2017-03-02 00:40:27.500000+00:00,127.40056,36.32481,127.39968,36.32704,1,1488382804,0
1,대전50바1004,2017-03-02 00:40:27.500000+00:00,2017-03-02 00:41:28.600000+00:00,127.39968,36.32704,127.40847,36.32965,1,1488382827,0
2,대전50바1004,2017-03-02 00:41:28.600000+00:00,2017-03-02 00:42:17.200000+00:00,127.40847,36.32965,127.41339,36.33458,1,1488382888,0
3,대전50바1004,2017-03-02 00:42:17.200000+00:00,2017-03-02 00:43:47.700000+00:00,127.41339,36.33458,127.42331,36.34238,1,1488382937,0
4,대전50바1004,2017-03-02 00:43:47.700000+00:00,2017-03-02 00:44:13.300000+00:00,127.42331,36.34238,127.42736,36.34369,1,1488383027,0
5,대전50바1004,2017-03-02 00:44:13.300000+00:00,2017-03-02 00:44:52.500000+00:00,127.42736,36.34369,127.43215,36.34740,1,1488383053,0
6,대전50바1004,2017-03-02 00:44:52.500000+00:00,2017-03-02 00:45:54.500000+00:00,127.43215,36.34740,127.43178,36.34739,1,1488383092,0
7,대전50바1004,2017-03-02 00:45:54.500000+00:00,2017-03-02 00:46:03.800000+00:00,127.43178,36.34739,127.43069,36.34646,1,1488383154,0
8,대전50바1004,2017-03-02 00:46:03.800000+00:00,2017-03-02 00:47:25.100000+00:00,127.43069,36.34646,127.42840,36.35002,1,1488383163,0
9,대전50바1004,2017-03-02 00:47:25.100000+00:00,2017-03-02 00:47:39.800000+00:00,127.42840,36.35002,127.42813,36.35148,1,1488383245,0


In [131]:
from tqdm import tqdm
new_df['trip'] = None

for i in tqdm(list(set(df['carnum']))):
    sample = new_df.loc[new_df['carnum']==i]
    trip_num = 1
    for trip in range(sample.shape[0]):
        if trip == 0:
            new_df['trip'][sample.index[trip]] = trip_num
        elif trip <= sample.shape[0] and sample['tpep_pickup_datetime'][sample.index[trip]] - sample['tpep_dropoff_datetime'][sample.index[trip-1]] > timedelta(minutes=1):
            trip_num = trip_num + 1
            new_df['trip'][sample.index[trip]] = trip_num
        else:
            new_df['trip'][sample.index[trip]] = trip_num

  0%|                                                                                          | 0/397 [00:00<?, ?it/s]<ipython-input-131-d62a397d169e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['trip'][sample.index[trip]] = trip_num
<ipython-input-131-d62a397d169e>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['trip'][sample.index[trip]] = trip_num
<ipython-input-131-d62a397d169e>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

In [135]:
import time
new_df['seconds'] = new_df['tpep_pickup_datetime'].apply(lambda x:time.mktime(x.timetuple()))
new_df['seconds'] = new_df['seconds'].astype(int)

In [146]:
from tqdm import tqdm
df['trip'] = None

for i in tqdm(list(set(df['carnum']))):
    sample = df.loc[df['carnum']==i]
    trip_num = 1
    for j in range(sample.shape[0]):
        df['trip'][sample.index[j]] = trip_num
        trip_num +=1

  0%|                                                                                          | 0/397 [00:00<?, ?it/s]<ipython-input-146-4572b0e73856>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['trip'][sample.index[j]] = trip_num
100%|███████████████████████████████████████████████████████████████████████████████| 397/397 [00:02<00:00, 155.41it/s]


In [154]:
sample

,carnum,tpep_pickup_datetime,tpep_dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip,seconds,altitude


In [166]:
sample = new_df.loc[(new_df['carnum']==df['carnum'][0])&(new_df['trip']==df['trip'][0])]
sample

,carnum,tpep_pickup_datetime,tpep_dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip,seconds,altitude
0,대전50바1004,2017-03-02 00:40:04+00:00,2017-03-02 00:40:27.500000+00:00,127.40056,36.32481,127.39968,36.32704,1,1488382804,None
1,대전50바1004,2017-03-02 00:40:27.500000+00:00,2017-03-02 00:41:28.600000+00:00,127.39968,36.32704,127.40847,36.32965,1,1488382827,None
2,대전50바1004,2017-03-02 00:41:28.600000+00:00,2017-03-02 00:42:17.200000+00:00,127.40847,36.32965,127.41339,36.33458,1,1488382888,None
3,대전50바1004,2017-03-02 00:42:17.200000+00:00,2017-03-02 00:43:47.700000+00:00,127.41339,36.33458,127.42331,36.34238,1,1488382937,None
4,대전50바1004,2017-03-02 00:43:47.700000+00:00,2017-03-02 00:44:13.300000+00:00,127.42331,36.34238,127.42736,36.34369,1,1488383027,None
5,대전50바1004,2017-03-02 00:44:13.300000+00:00,2017-03-02 00:44:52.500000+00:00,127.42736,36.34369,127.43215,36.34740,1,1488383053,None
6,대전50바1004,2017-03-02 00:44:52.500000+00:00,2017-03-02 00:45:54.500000+00:00,127.43215,36.34740,127.43178,36.34739,1,1488383092,None
7,대전50바1004,2017-03-02 00:45:54.500000+00:00,2017-03-02 00:46:03.800000+00:00,127.43178,36.34739,127.43069,36.34646,1,1488383154,None
8,대전50바1004,2017-03-02 00:46:03.800000+00:00,2017-03-02 00:47:25.100000+00:00,127.43069,36.34646,127.42840,36.35002,1,1488383163,None
9,대전50바1004,2017-03-02 00:47:25.100000+00:00,2017-03-02 00:47:39.800000+00:00,127.42840,36.35002,127.42813,36.35148,1,1488383245,None


In [177]:
#sample['tpep_pickup_datetime'][0]
sample['tpep_dropoff_datetime'][len(sample['tpep_dropoff_datetime'])-1]
#sample['tpep_pickup_datetime'][sample['tpep_pickup_datetime'].index[0]]

Timestamp('2017-03-02 00:48:00.400000+0000', tz='UTC')

In [179]:
sample['tpep_dropoff_datetime']

0    2017-03-02 00:40:27.500000+00:00
1    2017-03-02 00:41:28.600000+00:00
2    2017-03-02 00:42:17.200000+00:00
3    2017-03-02 00:43:47.700000+00:00
4    2017-03-02 00:44:13.300000+00:00
5    2017-03-02 00:44:52.500000+00:00
6    2017-03-02 00:45:54.500000+00:00
7    2017-03-02 00:46:03.800000+00:00
8    2017-03-02 00:47:25.100000+00:00
9    2017-03-02 00:47:39.800000+00:00
10   2017-03-02 00:48:00.400000+00:00
Name: tpep_dropoff_datetime, dtype: datetime64[ns, UTC]

In [201]:
new_df.loc[new_df['trip']==df['trip'][46]]

,carnum,tpep_pickup_datetime,tpep_dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip,seconds,altitude
1109,대전50바1009,2017-03-02 18:15:00+00:00,2017-03-02 18:16:36.700000+00:00,127.33962,36.37012,127.34007,36.37002,42,1488446100,0
1110,대전50바1009,2017-03-02 18:16:36.700000+00:00,2017-03-02 18:17:01.500000+00:00,127.34007,36.37002,127.33900,36.36669,42,1488446196,0
1111,대전50바1009,2017-03-02 18:17:01.500000+00:00,2017-03-02 18:17:19.400000+00:00,127.33900,36.36669,127.33809,36.36429,42,1488446221,0
1112,대전50바1009,2017-03-02 18:17:19.400000+00:00,2017-03-02 18:17:39.100000+00:00,127.33809,36.36429,127.34036,36.36353,42,1488446239,0
1113,대전50바1009,2017-03-02 18:17:39.100000+00:00,2017-03-02 18:18:26.800000+00:00,127.34036,36.36353,127.34411,36.36213,42,1488446259,0
...,...,...,...,...,...,...,...,...,...,...
101677,대전50바2328,2017-03-02 23:35:58.600000+00:00,2017-03-02 23:36:55.400000+00:00,127.43599,36.34079,127.43200,36.34786,42,1488465358,0
101678,대전50바2328,2017-03-02 23:36:55.400000+00:00,2017-03-02 23:37:58.600000+00:00,127.43200,36.34786,127.43284,36.35647,42,1488465415,0
101679,대전50바2328,2017-03-02 23:37:58.600000+00:00,2017-03-02 23:38:16.200000+00:00,127.43284,36.35647,127.43254,36.35885,42,1488465478,0
101680,대전50바2328,2017-03-02 23:38:16.200000+00:00,2017-03-02 23:38:20.400000+00:00,127.43254,36.35885,127.43214,36.35922,42,1488465496,0


In [200]:
new_df.loc[new_df['carnum']==df['carnum'][46]]

,carnum,tpep_pickup_datetime,tpep_dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip,seconds,altitude
55,대전50바1007,2017-03-02 00:07:35+00:00,2017-03-02 00:07:49.900000+00:00,127.37750,36.35670,127.37749,36.35763,1,1488380855,0
56,대전50바1007,2017-03-02 00:07:49.900000+00:00,2017-03-02 00:09:28.400000+00:00,127.37749,36.35763,127.38267,36.35769,1,1488380869,0
57,대전50바1007,2017-03-02 00:09:28.400000+00:00,2017-03-02 00:11:09.400000+00:00,127.38267,36.35769,127.39991,36.35763,1,1488380968,0
58,대전50바1007,2017-03-02 00:11:09.400000+00:00,2017-03-02 00:11:18.100000+00:00,127.39991,36.35763,127.40139,36.35757,1,1488381069,0
59,대전50바1007,2017-03-02 00:11:18.100000+00:00,2017-03-02 00:11:41.600000+00:00,127.40139,36.35757,127.40531,36.35684,1,1488381078,0
...,...,...,...,...,...,...,...,...,...,...
501,대전50바1007,2017-03-02 23:02:00.300000+00:00,2017-03-02 23:02:31.100000+00:00,127.44180,36.31744,127.44437,36.31634,41,1488463320,0
502,대전50바1007,2017-03-02 23:02:31.100000+00:00,2017-03-02 23:02:42.800000+00:00,127.44437,36.31634,127.44493,36.31528,41,1488463351,0
503,대전50바1007,2017-03-02 23:02:42.800000+00:00,2017-03-02 23:03:02.900000+00:00,127.44493,36.31528,127.44620,36.31572,41,1488463362,0
504,대전50바1007,2017-03-02 23:03:02.900000+00:00,2017-03-02 23:03:48.100000+00:00,127.44620,36.31572,127.44650,36.31433,41,1488463382,0


In [202]:
features = []
new_df['altitude'] = 0
for i in range(df.shape[0]):
    
    coordinates_list = []
    sample = new_df.loc[(new_df['carnum']==df['carnum'][i])&(new_df['trip']==df['trip'][i])]
    sample = sample.reset_index()
    if not sample.shape[0]==0:
        start_time = sample['tpep_pickup_datetime'][0]
        start_time = start_time.strftime('%Y-%m-%d-%H:%M:%S')
        end_time = sample['tpep_dropoff_datetime'][len(sample['tpep_dropoff_datetime'])-1]
        end_time = end_time.strftime('%Y-%m-%d-%H:%M:%S')
        for coordinates in zip(sample['pickup_longitude'],sample['pickup_latitude'],sample['altitude'],sample['seconds']):
                coordinates_list.append(list(coordinates))
        features.append(
        {
         "type": "Feature",
         "properties": {
            "start_time": start_time,
            "end_time": end_time,
            "trip_uuid": df['carnum'][i]
         },
        "geometry": {
          "type": "LineString",
          "coordinates": coordinates_list
         }
        }
        )        

In [204]:
new_df

,carnum,tpep_pickup_datetime,tpep_dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip,seconds,altitude
0,대전50바1004,2017-03-02 00:40:04+00:00,2017-03-02 00:40:27.500000+00:00,127.40056,36.32481,127.39968,36.32704,1,1488382804,0
1,대전50바1004,2017-03-02 00:40:27.500000+00:00,2017-03-02 00:41:28.600000+00:00,127.39968,36.32704,127.40847,36.32965,1,1488382827,0
2,대전50바1004,2017-03-02 00:41:28.600000+00:00,2017-03-02 00:42:17.200000+00:00,127.40847,36.32965,127.41339,36.33458,1,1488382888,0
3,대전50바1004,2017-03-02 00:42:17.200000+00:00,2017-03-02 00:43:47.700000+00:00,127.41339,36.33458,127.42331,36.34238,1,1488382937,0
4,대전50바1004,2017-03-02 00:43:47.700000+00:00,2017-03-02 00:44:13.300000+00:00,127.42331,36.34238,127.42736,36.34369,1,1488383027,0
...,...,...,...,...,...,...,...,...,...,...
105080,대전50바2421,2017-03-02 19:32:25.200000+00:00,2017-03-02 19:33:21.900000+00:00,127.40163,36.35127,127.40179,36.34475,23,1488450745,0
105081,대전50바2421,2017-03-02 19:33:21.900000+00:00,2017-03-02 19:33:47.200000+00:00,127.40179,36.34475,127.40491,36.34454,23,1488450801,0
105082,대전50바2421,2017-03-02 19:33:47.200000+00:00,2017-03-02 19:34:06+00:00,127.40491,36.34454,127.40698,36.34369,23,1488450827,0
105083,대전50바2421,2017-03-02 19:34:06+00:00,2017-03-02 19:37:51.400000+00:00,127.40698,36.34369,127.42586,36.32864,23,1488450846,0


In [205]:
features

[{'type': 'Feature',
  'properties': {'start_time': '2017-03-02-00:40:04',
   'end_time': '2017-03-02-00:48:00',
   'trip_uuid': '대전50바1004'},
  'geometry': {'type': 'LineString',
   'coordinates': [[127.40056, 36.32481, 0, 1488382804],
    [127.39968, 36.32704, 0, 1488382827],
    [127.40847, 36.32965, 0, 1488382888],
    [127.41339, 36.33458, 0, 1488382937],
    [127.42331, 36.34238, 0, 1488383027],
    [127.42736, 36.34369, 0, 1488383053],
    [127.43215, 36.3474, 0, 1488383092],
    [127.43178, 36.34739, 0, 1488383154],
    [127.43069, 36.34646, 0, 1488383163],
    [127.4284, 36.35002, 0, 1488383245],
    [127.42813, 36.35148, 0, 1488383259]]}},
 {'type': 'Feature',
  'properties': {'start_time': '2017-03-02-00:50:02',
   'end_time': '2017-03-02-00:57:34',
   'trip_uuid': '대전50바1004'},
  'geometry': {'type': 'LineString',
   'coordinates': [[127.42618, 36.35116, 0, 1488383402],
    [127.42813, 36.35148, 0, 1488383422],
    [127.4284, 36.35002, 0, 1488383437],
    [127.42948, 36.348

In [203]:
import json
with open('C:/Users/user/Desktop/택시이동경로/total.json', 'w') as f:
    json.dump(features, f)

In [ ]:
!pip install keplergl
!pip install geopandas
!pip install sodapy

In [209]:
feature = gpd.GeoDataFrame(features)